In [1]:
from google.colab import userdata
token = userdata.get('HF_TOKEN')

In [2]:
from huggingface_hub import login
login(token=token)

In [3]:
from transformers import pipeline

In [4]:
# pipe = pipeline("text-generation", model="google/gemma-3-1b-it")
# pipe = pipeline("text-generation", model="google/gemma-3-4b-it")
# messages= [{"role":"user","content":"What's Python?"}]
# response = pipe(messages)
# print(response[0]["generated_text"][1]["content"])

In [ ]:
# !pip install langchain-huggingface

In [6]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.messages import HumanMessage, SystemMessage
from transformers import BitsAndBytesConfig

llm = HuggingFacePipeline.from_model_id(
    model_id="google/gemma-3-1b-it",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 5120,
        "temperature": 0.8,}
)

chat_model = ChatHuggingFace(llm=llm)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
messages = [SystemMessage("你是一個寫五言絕句，的寫詩的大師，請用繁體中文"),
            HumanMessage("請寫一首關於PYTHON的詩")]

response = chat_model.invoke(messages)
print(response.content)

<bos><start_of_turn>user
你是一個寫五言絕句，的寫詩的大師，請用繁體中文

請寫一首關於PYTHON的詩<end_of_turn>
<start_of_turn>model
```python
夜幕星光閃，
一行代码悄然落。
數據流淌去，
智能深思量。
```

**解析：**

*   **夜幕星光閃：** 描寫了Python在夜晚的樣子，營造出神秘、閃爍的氛圍。
*   **一行代码悄然落：** 點明主題，Python程式碼。
*   **數據流淌去：**  將Python程式碼的運行過程，代表數據的流動。
*   **智能深思量：**  表達Python的智慧和潛力，暗示其能力。

希望你喜歡！



In [3]:
# from langchain_community.document_loaders import PyPDFLoader
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd drive/MyDrive/data_rag

/content/drive/MyDrive/data_rag


In [ ]:
# !pip install langchain_community pypdf

In [5]:
from langchain_community.document_loaders import PyPDFLoader


In [6]:
loader = PyPDFLoader("2509.05396v1.pdf")

In [7]:
doc = loader.load()

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader

from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.messages import HumanMessage, SystemMessage
from transformers import BitsAndBytesConfig

llm = HuggingFacePipeline.from_model_id(
    model_id="google/gemma-3-1b-it",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 5120,
        "temperature": 0.8,}
)

chat_model = ChatHuggingFace(llm=llm)

loader = PyPDFLoader("2509.05396v1.pdf")
doc = loader.load()

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
)

# 1. 分割文件
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

texts = text_splitter.split_documents(doc)
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

rag_prompt = PromptTemplate(
    template="""根據以下資訊回答問題。如果資訊中沒有相關內容, 請說你不知道。

相關資訊:
{context}

問題: {question}

回答:""",
    input_variables=["context", "question"]
)

# 6. 建立 RAG Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": rag_prompt},
    return_source_documents=True
)



Device set to use cuda:0


In [ ]:
# !pip install -U faiss-cpu==1.10.0

In [20]:
result = qa_chain.invoke({"query": "請問 multi agent debate 有沒有用?"})

In [21]:
result["query"]

'請問 multi agent debate 有沒有用?'

In [22]:
print(result["result"])

<bos><start_of_turn>user
根據以下資訊回答問題。如果資訊中沒有相關內容, 請說你不知道。

相關資訊:
Talk Isn’t Always Cheap: Understanding Failure Modes in Multi-Agent Debate
Andrea Wynn* 1 Harsh Satija * 2 Gillian Hadfield 1 3 2
Abstract
While multi-agent debate has been proposed as
a promising strategy for improving AI reason-
ing ability, we find that debate can sometimes be
harmful rather than helpful. The prior work has
exclusively focused on debates within homoge-
neous groups of agents, whereas we explore how
diversity in model capabilities influences the dy-

tions (Irving et al., 2018; Khan et al., 2024; Michael et al.,
2023; Kenton et al., 2024). More recently, another form of
multi-agent debate, sometimes referred to as multi-agent
deliberation, investigates at leveraging different LLM agents
to surface better answers by having them exchange reason-
ing via iterative discussion (Du et al., 2023; Chan et al.,
2023; Liang et al., 2023; Subramaniam et al., 2025). Most
of these studies focus on a homogeneous setti